# Tutorial 5 -- Human Detection 
This tutorial will show you how to detect human with the robot. We will use multi-thread to capture the color images and depth images. Then the ssd mobilenet v2 nerual network structure is used for human detection. To enable real-time performance, the tensorRT deep learning framework is used. More information can be found in the nvidia's jetbot project website: https://github.com/NVIDIA-AI-IOT/jetbot

### Step 1 load the pre-trained object detection model

**Before running the program, we need to download the pre-trained detection model and put it in the current folder.** The model can be found in the following link:https://drive.google.com/file/d/1KjlDMRD8uhgQmQK-nC2CZGHFTbq4qQQH/view

In [1]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')


### Step 2 Initialize the camera instance for the Intel realsense sensor D435i

In [2]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


### Step 3 Perform object detection and display the results on widgets
Human is labeled is 1 in the pretrained model. A full list of the detection class indices can be found in the following link https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt

The following program will ask the robot to move forward if a human is detected in the captured color image.

In [3]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML

width = 640
height = 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
display(widgets.VBox([
    widgets.HBox([image_widget,]),
    label_widget
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

# Initialize the tracker
tracker = cv2.TrackerMOSSE_create()
tracker_initialized = False
firstitter = False


def processing(change):
    global tracker_initialized, tracker, bbox, firstitter
    image = change['new']
    imageTrack = image.astype("float32") /255
     
    imgsized= cv2.resize(image,(300,300))
    
    # compute all detected objects
    detections = model(imgsized)

    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]

    '''
    indev = 0

    for det in range(matching_detections):
        if depth_value(matching_detections[det]['bbox']) < depth_value(matching_detections[index]['bbox']):
            index = det
    '''
    if not firstitter:
        old_bbox = (0,0,width, height)
        firstiter = True

    for det in matching_detections:
        new_bbox = det['bbox']
        if (old_bbox[0] <= int(width * (new_bbox[0] + new_bbox[2])) <= old_bbox[0] + old_bbox[2]) or (old_bbox[0] <= int(width * new_bbox[0]) <= old_bbox[0] + old_bbox[2]):
            if #add height
            bbox = det['bbox']
            bbox = (int(width * bbox[0]), int(height * bbox[1]), int(width * bbox[2]), int(height * bbox[3]))
            old_bbox = bbox

    #need to fix V
    x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
    #
    center_x = w + x / 2.0
    center_y = h + y / 2.0

    # Determine the midpoint of the frame's width
    frame_midpoint = image.shape[1] // 2
    # Calculate 10% of the frame's width to define the center zone
    center_zone_width = image.shape[1] * 0.1
    left_center_zone = frame_midpoint - (center_zone_width / 2)
    right_center_zone = frame_midpoint + (center_zone_width / 2)

    # Check if the center point is near the left or right edge or has moved off-frame
    if center_x < 10:
        print("Action: Near left edge or moved off-frame to the left")
        #robot.left(0.3)  # Turn left
    elif center_x > image.shape[1] - 10:
        print("Action: Near right edge or moved off-frame to the right")
        #robot.right(0.3)  # Turn right

    # Check if the center point is within the center zone
    elif left_center_zone <= center_x <= right_center_zone:
        print("Action: In the center zone of the frame")
        #robot.forward(0.3)  # Move forward if roughly centered
    elif center_x < frame_midpoint:
        print("Action: In the left half of the frame")
        #robot.forward_left(0.3)  # Turn left
    elif center_x > frame_midpoint:
        print("Action: In the right half of the frame")
        #robot.forward_right(0.3)  # Turn right

    print(x,y,w,h)
    # Draw the updated bounding box and center point
    x_int = int(x)
    y_int = int(y)
    w_int = int(w)
    h_int = int(h)

    cv2.rectangle(image, (x_int, y_int), (x_int + w_int, y_int + h_int), (255, 0, 0), 2)


    #cv2.circle(image, int(center_x, center_y), 5, (0, 255, 0), -1)  # Draw center point

        
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

Action: Near right edge or moved off-frame to the right
583 185 639 396
Action: Near right edge or moved off-frame to the right
584 187 639 391
Action: Near right edge or moved off-frame to the right
585 188 640 384
Action: Near right edge or moved off-frame to the right
584 188 639 382
Action: Near right edge or moved off-frame to the right
584 191 640 383
Action: In the right half of the frame
166 10 291 419
Action: Near right edge or moved off-frame to the right
461 196 530 399
Action: In the right half of the frame
166 14 290 415
Action: Near right edge or moved off-frame to the right
439 265 486 403
Action: Near right edge or moved off-frame to the right
439 266 486 401
Action: In the right half of the frame
164 17 289 412
Action: Near right edge or moved off-frame to the right
579 186 638 386
Action: Near right edge or moved off-frame to the right
436 267 485 402
Action: Near right edge or moved off-frame to the right
460 269 524 401
Action: Near right edge or moved off-frame to 

In [4]:
camera.unobserve_all()
camera.stop()
time.sleep(1.0)
robot.stop()

Action: In the right half of the frame
318 278 401 400
Action: In the right half of the frame
317 279 401 398


## Tasks

### 1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to tutorial 2 to get the depth information for a specific point)
### 2. Please try to add collision avoidance function into this program to protect the robot. (Note: You can refer to tutorial 4 for the collision avoidance)
### 3. Think about how to control the robot to move towards the detected human

NameError: name 'robot' is not defined